# Python & Bioinformatics
## Advanced Python for Life Sciences @ Physalia courses (Summer 2025)
### Marco Chierici, Fondazione Bruno Kessler

In [1]:
# we use the pathlib library to improve file/folder manipulation
from pathlib import Path

DATADIR = Path("data")
OUTDIR = Path("out")

# Biopython

Biopython is a collections of tools for biological computation. Some of its features include:

- A standard sequence class that deals with sequences, ids on sequences, and sequence features.
- Tools for performing common operations on sequences, such as translation, transcription and weight calculations.
- Code to perform classification of data using k Nearest Neighbors, Naive Bayes or Support Vector Machines.
- Code for dealing with alignments, including a standard way to create and deal with substitution matrices.

More info and full documentation: https://biopython.org/

Install: just copy-paste the following into the empty cell below

```
!conda install -y -c conda-forge biopython
```

(Reminder: the `!` at the beginning of a Jupyter code cell allows you to run what follows as a terminal command)

## Basics

In [3]:
from Bio.Seq import Seq

my_seq = Seq("AGTACACTGGT")
my_seq

Seq('AGTACACTGGT')

For most aspects, a `Seq` objects acts like a normal Python string: you can compute its `len()` and slice it.

In [4]:
my_seq = Seq("GATCGATGGGCCTATATAGGATCGAAAATCGC")
len(my_seq)

32

In [5]:
my_seq_str = str(my_seq)
print(my_seq_str)
len(my_seq_str)

GATCGATGGGCCTATATAGGATCGAAAATCGC


32

In [6]:
print(my_seq[6:9])

TGG


In [7]:
print(my_seq.count("G"))
print(my_seq_str.count("G"))

9
9


### (Reverse) complement

In [9]:
txt = "Hello World" [::-1]
print(txt) 

dlroW olleH


In [11]:
r_comp = my_seq.reverse_complement()
print(r_comp)

GCGATTTTCGATCCTATATAGGCCCATCGATC


### Transcription, back transcription

Assuming `my_seq` is a DNA sequence, we may want to turn it into RNA.
We normally assume the DNA is the coding strand (not the template strand) so this is a simple matter of replacing all the thymines with uracil:

In [14]:
print(my_seq)
print(my_seq.transcribe())

GATCGATGGGCCTATATAGGATCGAAAATCGC
GAUCGAUGGGCCUAUAUAGGAUCGAAAAUCGC


Given some RNA, you might want the associated DNA - again, a simple U/T substitution:

In [18]:
rna = my_seq.transcribe()
dna = rna.back_transcribe()
print(rna)
print(dna)

GAUCGAUGGGCCUAUAUAGGAUCGAAAAUCGC
GATCGATGGGCCTATATAGGATCGAAAATCGC


If you actually do want the template strand, you’d have to do a reverse complement on top:

### Translation

You can translate RNA:

In [ ]:
mrna = Seq("AUGGCCAUUGUAAUGGGCCGCUGAAAGGGUGCCCGAUAG")
mrna.translate()

You can also translate DNA, which is assumed to be the coding strand:

In [ ]:
coding_dna = Seq("ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG")
coding_dna.translate()

We can provide `translate()` with a few options. By default, as you have noticed, translation continues through any stop codons, but this can be modified:

In [ ]:
# prevent translation to continue through stop codons
coding_dna.translate(to_stop=True)

Another option is the translation table, for which you can pass a [NCBI genetic code number](http://www.ncbi.nlm.nih.gov/Taxonomy/Utils/wprintgc.cgi) or name:

In [ ]:
# specify a translation table (NCBI genetic code number)
coding_dna.translate(table=2)

In [ ]:
coding_dna.translate(table="Vertebrate Mitochondrial")

In [ ]:
# combining options
coding_dna.translate(table=2, to_stop=True)

You can also change the symbol for the stop codon with the option `stop_symbol`.

Note that ambiguous codons like "TAN" or "NNN" could be an amino acid
or a stop codon: these are translated as "X".

In [24]:
Seq("ATGGCCATTGTATANGTAATG").translate(to_stop=True)
##### TRANSLATE DOESN'T STOP AT X

Seq('MAIVXVM')

In principle, Biopython does not know if a `Seq` object represents DNA, RNA, or a protein. Suppose your sequence contains valid nucleotide *and* amino acid letters and you want to compute the molecular weight calling the method `molecular_weight` on the sequence. If you know what your `Seq` object represents, then you have to tell Biopython:

In [ ]:
from Bio.SeqUtils import molecular_weight

my_seq = Seq("AGTACACTGGT")
molecular_weight(my_seq)

In [ ]:
molecular_weight(my_seq, "protein")

## SeqRecord

`SeqRecord` objects are very similar to `Seq` objects, but they have also a few additional attributes:

- seq: The sequence itself, typically a Seq object.
- id: The primary ID used to identify the sequence – a string. In most cases this is something like an accession number.
- name: A common name/id for the sequence – a string. In some cases this will be the same as the accession number, but it could also be a clone name. Analagous to the LOCUS id in a GenBank record.
- description: A human readable description or expressive name for the sequence – a string.

`SeqRecord` is the basis of `SeqIO` objects, which handle file I/O in different biological formats.

In [ ]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

test_seq = Seq("GATCAGGATTAGGCC")
test_record = SeqRecord(test_seq, id="xyz")
test_record.description = "I am not a real sequence"
print(test_record)

In [ ]:
print(test_record.description)

In [ ]:
print(test_record.seq)

## SeqIO

The `SeqIO` module reads or writes sequences as `SeqRecord` objects. It is like a container for multiple `SeqRecord`s.

`SeqIO` handles many different file formats, including FASTA, FASTQ (Sanger, Solexa, Illumina), GenBank, SFF (IonTorrent/IonProton), UniProt/SwissProt, EMBL, Clustal alignments, and more.

In [25]:
from Bio import SeqIO
with open(DATADIR / "database.fasta") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        print(record.id)

seq0
seq1
seq2
seq3
seq4
seq5
seq6
seq7
seq8
seq9
seq10


Note that `SeqIO.parse()` does not guess the file format by extension or content: you must specify the format explicitly. This is also compliant with the Zen of Python "Explicit is better than implicit".

If you had a different file type, for example a GenBank file, you just specify 'genbank' instead of 'fasta' in the parser:

In [26]:
from Bio import SeqIO
with open(DATADIR / "cor6_6.gb") as handle:
    for record in SeqIO.parse(handle, "genbank"):
        print(record.id)

X55053.1
X62281.1
M81224.1
AJ237582.1
L31939.1
AF297471.1


`SeqIO.parse()` is an *iterator* - which is great when you only need the records one by one, in the order found in the file.

For some tasks you may need to have random access to the records in any order. In this situation, turn the iterator into a list:

In [30]:
records = list(SeqIO.parse(DATADIR / "database.fasta", "fasta"))
print(records[0].id)  # first record
print(records[-1].id)  # last record
print(records[-1])  # last record

seq0
seq10
ID: seq10
Name: seq10
Description: seq10
Number of features: 0
Seq('FDSWDEFVSKSVELFRNHPDTTRYVVKYRHCEGKLVLKVTDNHECLKFKTDQAQ...MEK')


If you have a bunch of sequences in a list or `SeqRecord` object, you can write them to a file by using `SeqIO.write()`.

In [28]:
# create the output dir and don't complain if it already exists
OUTDIR.mkdir(exist_ok=True)
# just write the first 2 records
SeqIO.write(records[:2], OUTDIR / "db_out.fasta", "fasta")

2

Another common task is to index your records by some identifier. For small files there's the function `SeqIO.to_dict()` to turn a SeqRecord iterator (or list) into a dictionary (in memory):

In [29]:
record_dict = SeqIO.to_dict(SeqIO.parse(DATADIR / "database.fasta", "fasta"))
print(record_dict["seq3"])  # use any record ID

ID: seq3
Name: seq3
Description: seq3
Number of features: 0
Seq('MYQVWEEFSRAVEKLYLTDPMKVRVVLKYRHCDGNLCIKVTDNSVCLQYKTDQAQDVK')


For larger files, where you can't hold everything in memory, you can use `SeqIO.index()`:

In [31]:
record_dict = SeqIO.index(str(DATADIR / "database.fasta"), "fasta")
print(record_dict["seq3"])  # use any record ID

ID: seq3
Name: seq3
Description: seq3
Number of features: 0
Seq('MYQVWEEFSRAVEKLYLTDPMKVRVVLKYRHCDGNLCIKVTDNSVCLQYKTDQAQDVK')


### I/O: format conversion

To convert between file formats (e.g., GenBank -> FASTA), there is the convenient `SeqIO.convert()` function.

In [32]:
count = SeqIO.convert(DATADIR/"cor6_6.gb", "genbank", OUTDIR/"cor6_6.fasta", "fasta")
print(f"Converted {count} records")

Converted 6 records


### Sequence features

We read now a Genbank file with a whole mitochondrial genome (e.g. the tobacco mitochondrion, *Nicotiana tabacum* mitochondrion NC_006581).

In [33]:
mito_record = SeqIO.read(DATADIR/"NC_006581.gbk", "genbank")

In [34]:
print(mito_record.id)
print(len(mito_record))
print(len(mito_record.features))

NC_006581.1
430597
409


What is this `mito_record.features` object?

It's a Python list, containing a Biopython `SeqFeature` object for each feature in the GenBank file. For instance:

In [49]:
mito_record.features

[SeqFeature(SimpleLocation(ExactPosition(0), ExactPosition(430597), strand=1), type='source', qualifiers=...),
 SeqFeature(SimpleLocation(ExactPosition(824), ExactPosition(1202), strand=1), type='gene', qualifiers=...),
 SeqFeature(SimpleLocation(ExactPosition(824), ExactPosition(1202), strand=1), type='CDS', qualifiers=...),
 SeqFeature(SimpleLocation(ExactPosition(2386), ExactPosition(2803), strand=1), type='gene', qualifiers=...),
 SeqFeature(SimpleLocation(ExactPosition(2386), ExactPosition(2803), strand=1), type='CDS', qualifiers=...),
 SeqFeature(SimpleLocation(ExactPosition(4035), ExactPosition(6012), strand=-1), type='gene', qualifiers=...),
 SeqFeature(SimpleLocation(ExactPosition(4035), ExactPosition(6012), strand=-1), type='CDS', qualifiers=...),
 SeqFeature(CompoundLocation([SimpleLocation(ExactPosition(288447), ExactPosition(288832), strand=1), SimpleLocation(ExactPosition(142902), ExactPosition(144389), strand=-1), SimpleLocation(ExactPosition(6673), ExactPosition(44807),

In [36]:
my_gene = mito_record.features[3]
print(my_gene)

type: gene
location: [2386:2803](+)
qualifiers:
    Key: db_xref, Value: ['GeneID:3205235']
    Key: gene, Value: ['orf138a']
    Key: locus_tag, Value: ['NitaMp003']



Notice the three key properties:
- `.type` which is a string like gene or CDS;
- `.location` which describes where on the genome this feature is;
- `.qualifiers` which is a Python dictionary with all the annotation for the feature (things like gene identifiers)

Here's how this gene looks like in the original GenBank file:

```
     gene            2387..2803
                     /gene="orf138a"
                     /locus_tag="NitaMp003"
                     /db_xref="GeneID:3205235"
```

In [37]:
my_gene.location

SimpleLocation(ExactPosition(2386), ExactPosition(2803), strand=1)

In [38]:
my_gene.location.strand

1

In [40]:
my_gene.location.start

ExactPosition(2386)

In [ ]:
gene_seq = mito_record.seq[ ... ]
print(len(gene_seq))
print(gene_seq)

This was a very simple location on the forward strand, if it had been on the reverse strand you'd need to take the reverse-complement.

Also if the location had been a more complicated compound location like a join (used for eukaryotic genes where the CDS is made up of several exons), then the location would have-sub parts to consider.

In [42]:
another_gene = mito_record.features[10]
print(another_gene)

type: gene
location: [7186:7702](-)
qualifiers:
    Key: db_xref, Value: ['GeneID:3205319']
    Key: gene, Value: ['orf171a']
    Key: locus_tag, Value: ['NitaMp007']



In [43]:
another_seq = mito_record.seq[ another_gene.location.start:another_gene.location.end ]
print(len(another_seq))
print(another_seq.reverse_complement())

516
ATGTGCTTTTATTATATTTGCCTGAAACTCGATAAAATAGAGTTGTTTGAGACCAAATGTTATAGAGTTGGGGGCTCTCTGGGGAGGCACACCTTCACTTCCTTCTTAATGAAGGGGTTCCCCGGGAGTCTGACCTTGTCCTTTCTTTTAATTGTCGGGGCCGTCATGAGTGCCGAAACCGATTTCGGTAAGATGATGATGGCTCCTTCGGGCGCATCAAGCTCTGAAGATCCAAACTGGACGGAAGCCCTGAGATCTTCTAAAGGGCAGGGAGAGACTTCAGAAAGGGAAAGCACAGGCACATCGTCGTCCATCAACCTACAAAAAGAAAGAGCACGTCCGGCGCCTGCCCCAAATGAAGTAGCTTCCCCTGCCCCTGTCGTCCCCTTTCCATATCAAGAAGATGAGATCATAGGGGGCGACAGTGTAGAAAGCATCCAACAGCGGCCTTTGAGTTTGAGGAGAAAAACCCCTCCTTCTGCCGAGGTCATACATCATACAACAGGCCCGAATTGA


All these complications are taken care of for you via the `.extract()` method which takes the full length parent record's sequence as an argument:

In [44]:
my_gene

SeqFeature(SimpleLocation(ExactPosition(2386), ExactPosition(2803), strand=1), type='gene', qualifiers=...)

In [45]:
gene_seq = my_gene.extract(mito_record.seq)
print(len(gene_seq))
print(gene_seq)

417
ATGTTAAAATCGAATCATCGACAAGATGTTTATCAGAGAGATTTTATTATGGGATACAATCGAAGGGCTCATAAAATGAAAAACTTGAGAAGTAAGGGAATATCGATGGCCTTTGTTGAACAGAGAGCGAACGGACGGCAAGTGACTTTGATCTCATGCTGGCCTTTACTTCGGCTACTAATAAGGGGGTTGGGGTTTGACAAATCAAGACTGGGGCTTCGTTTAGCCGCCTATGTAGTAAGAAAAACTCTGAGTTTGACGTTGAATAGCCATAGTGAGTTCATCACCAGTGGCATAAGCAAAGGAGGCATATGCGGTGGGGAGAAAGGCAATTTACCTCACTTCTTACAATTCCAGTTGAACCCGCAACAAAGCCAGCGTTGCTTTCGTGGGATCAACTGCTCGTTTTCGATATAG


### Exercise: Translating CDS features

When dealing with GenBank files and trying to get the protein sequence of the genes, you'll need to look at the CDS features (coding sequences) - not the gene features (although for simple cases they'll have the same location).

Sometimes you will find the translation is provided in the qualifiers. Read the *Nicotiana tabacum* mitochondrion NC_006581.gbk with `SeqIO` and print the translation by accessing the record qualifiers.

In [54]:
record = SeqIO.read(DATADIR/"NC_006581.gbk", "genbank")
#print(record)
my_cds = record.features[2]
print(my_cds.qualifiers["locus_tag"])
print(my_cds.qualifiers["translation"])


['NitaMp002']
['MRNTCKSRLKDKEGKSKRKESLSRSSSQRSKNSKAKSGLRHSNCMNVNLIYVTGLMIANAHKTNIITFFAGRRKEKTSETKWSEGEGGSTWNWLSVNPKYDIDRGKKISYMPKVHLRPSFQATMT']


In [23]:
# desired output

['NitaMp002']
['MRNTCKSRLKDKEGKSKRKESLSRSSSQRSKNSKAKSGLRHSNCMNVNLIYVTGLMIANAHKTNIITFFAGRRKEKTSETKWSEGEGGSTWNWLSVNPKYDIDRGKKISYMPKVHLRPSFQATMT']


However, many times the annotation will not include the amino acid translation - but we can get it by translating the nucleotide sequence. Now, manually translate the nucleotide sequence.

In [59]:
# extract the CDS from the parent record
cds_seq = my_cds.extract(record.seq)
# translate
protein_seq = cds_seq.translate()
# print the translated sequence with its length
print(len(protein_seq))
print(protein_seq)

126
MRNTCKSRLKDKEGKSKRKESLSRSSSQRSKNSKAKSGLRHSNCMNVNLIYVTGLMIANAHKTNIITFFAGRRKEKTSETKWSEGEGGSTWNWLSVNPKYDIDRGKKISYMPKVHLRPSFQATMT*


In [24]:
# desired output

126
MRNTCKSRLKDKEGKSKRKESLSRSSSQRSKNSKAKSGLRHSNCMNVNLIYVTGLMIANAHKTNIITFFAGRRKEKTSETKWSEGEGGSTWNWLSVNPKYDIDRGKKISYMPKVHLRPSFQATMT*


---

## Online databases

### Entrez

In [56]:
from Bio import Entrez

Entrez.email = "zhipei.du@gmail.com"  # use your email :)

To search any of the Entrez databases ("nucleotide", "protein", "pubmed"), we can use the `Bio.Entrez.esearch()` module with the following syntax:

```
handle = Entrez.esearch(db="value", term="keywords", retmax=100)
```

In [57]:
handle = Entrez.esearch(db="nucleotide", term="hemoglobin AND alpha", retmax=10)
records = Entrez.read(handle)
records

{'Count': '55172', 'RetMax': '10', 'RetStart': '0', 'IdList': ['3007250012', '3007242769', '3007240035', '3007231627', '3007228799', '3007226228', '3007223540', '3007214200', '3007213618', '3007193993'], 'TranslationSet': [], 'TranslationStack': [{'Term': 'hemoglobin[All Fields]', 'Field': 'All Fields', 'Count': '1', 'Explode': 'N'}, {'Term': 'alpha[All Fields]', 'Field': 'All Fields', 'Count': '1', 'Explode': 'N'}, 'AND'], 'QueryTranslation': 'hemoglobin[All Fields] AND alpha[All Fields]'}

In [58]:
identifiers = records["IdList"]
identifiers

['3007250012', '3007242769', '3007240035', '3007231627', '3007228799', '3007226228', '3007223540', '3007214200', '3007213618', '3007193993']

We can retrieve entries for a number of identifiers using `Bio.Entrez.efetch()`, which allows us to specify the return type (e.g. FASTA, GenBank).

```
handle = Entrex.efetch(db, id, rettype, retmode, retmax)
```

(for an explanation of the parameters, please see http://www.ncbi.nlm.nih.gov/books/NBK25499/#chapter4.EFetch)

In [ ]:
id = identifiers[3]
handle = Entrez.efetch(db="nucleotide", id=id,
                       rettype="fasta", retmode="text")
record = SeqIO.read(handle, "fasta")
print(record)

To search for papers on PubMed:

In [ ]:
handle = Entrez.esearch(db="pubmed", term="thermophilic,packing", rettype="uilist", retmax=10)
records = Entrez.read(handle)
records

To retrieve a publication record in Medline format:

In [ ]:
from Bio import Medline
handle = Entrez.efetch(db="pubmed", id="37837765,37833340",
                       rettype="medline", retmode="text")
records = Medline.parse(handle)
for record in records:
    print(f"{record['TI']} ({record['DP']})")


To search for protein database entries by keyword:

In [ ]:
handle = Entrez.esearch(db="protein", term="cancer AND human")
records = Entrez.read(handle)
records

To retrieve protein database entries in FASTA format:

In [ ]:
handle = Entrez.efetch(db="protein", id="2619400481",
                       rettype="fasta", retmode="text")
record = SeqIO.read(handle, "fasta")
print(record)

To retrieve protein database entries in Genbank format:

In [ ]:
handle = Entrez.efetch(db="protein", id="2619400481",
                       rettype="gb", retmode="text")
record = SeqIO.read(handle, "genbank")
print(record)

### BLAST

In [ ]:
from Bio.Blast import NCBIWWW

NCBIWWW.email = ""  # insert your email here :)

We can perform a BLAST search online using the `NCBIWWW.qblast()` function:

```
output = NCBIWWW.qblast(program, database, sequence)
```

`program`: the BLAST program to use for the search; blastn, blastp, blastx, tblastn, tblastx
`database`: the BLAST database to search against (e.g. 'nt', 'nr')
`sequence`: a FASTA file, a SeqIO record, or a Genbank accession number.

In the following example, we'll use the GI accession number 8332116: this is a DNA sequence so we choose blastn. If you would like to use a protein sequence, use blastp.

In [ ]:
# not run:
# result_handle = NCBIWWW.qblast("blastn", "nt", "8332116")
# with open(DATADIR / "my_blast.xml", "w") as out_handle:
#     out_handle.write(result_handle.read())
# result_handle.close()

In [ ]:
# other possibilities:
# 1. from FASTA
# sequence_data = open(DATADIR / "m_cold.fasta").read()
# result_handle = NCBIWWW.qblast("blastn", "nt", sequence_data)
# ...
# 2. from FASTA as SeqRecord
# record = SeqIO.read("m_cold.fasta", format="fasta")
# result_handle = NCBIWWW.qblast("blastn", "nt", record.seq)

In [ ]:
from Bio.Blast import NCBIXML
result_handle = open(DATADIR / "my_blast.xml")
blast_records = NCBIXML.parse(result_handle)
blast_record = next(blast_records)

Here, `my_blast.xml` contains several records (because we used more than one query sequence): thus, we read it using `NCBIXML.parse()` and then iterate over the returned records.

You can get the alignments from `blast_record.alignments`:

In [ ]:
blast_record.alignments[:10]

In [ ]:
print("Alignments for sequence", blast_record.query)

for alignment in blast_record.alignments[:10]:
    print("Accession number:", alignment.accession)
    print("Sequence:", alignment.title)
    print("Length:", alignment.length)
    print()

In [ ]:
result_handle = open(DATADIR / "my_blast.xml")
blast_records = NCBIXML.parse(result_handle)
blast_record = next(blast_records)

E_VALUE_THRESH = 0.04

for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.expect < E_VALUE_THRESH:
            print("****Alignment****")
            print("sequence:", alignment.title)
            print("length:", alignment.length)
            print("e value:", hsp.expect)
            print(hsp.query[0:75] + "...")
            print(hsp.match[0:75] + "...")
            print(hsp.sbjct[0:75] + "...")
            print()

In [ ]:
result_handle = open(DATADIR / "my_blast.xml")
blast_records = NCBIXML.parse(result_handle)

E_VALUE_THRESH = 0.04

for blast_record in blast_records:
    if blast_record.alignments:
        print(f"QUERY: {blast_record.query[:60]}...")
        for alignment in blast_record.alignments:
            for hsp in alignment.hsps:
                if hsp.expect < E_VALUE_THRESH:
                    print(f"MATCH: {alignment.title[:60]}...")
                    print("e value:", hsp.expect)
                    print()

---

# Credits

Partially abridged from a number of sources, including BioPython documentation, Peter Cock (CC-BY-SA 4.0), Kuan Rong Chan.